In [7]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cufflinks as cf
import plotly.subplots as sp

# Configuring cufflinks for offline mode
cf.go_offline()

In [3]:
# Download historical data for S&P 500 index
sp500 = yf.download('^GSPC', start='1920-01-01', end='2030-12-31')

# Resample to monthly data
monthly_sp500 = sp500.resample('M').last()


[*********************100%***********************]  1 of 1 completed


In [4]:
cycles = {
    'Initial Bottom': {'years': [1924, 1942, 1958, 1978, 1996, 2012], 'color': 'green'},
    'Top': {'years': [1927, 1945, 1965, 1981, 1999, 2019], 'color': 'red'},
    'Bottom': {'years': [1931, 1951, 1969, 1985, 2005, 2023], 'color': 'green'},
    'Mid Top': {'years': [1935, 1953, 1972, 1989, 2007, 2026], 'color': 'red'}
}

In [8]:
# Calculate Moving Average and Bollinger Bands
monthly_sp500['MA'] = monthly_sp500['Close'].rolling(window=20).mean()
monthly_sp500['upper_band'] = monthly_sp500['MA'] + 2*monthly_sp500['Close'].rolling(window=20).std()
monthly_sp500['lower_band'] = monthly_sp500['MA'] - 2*monthly_sp500['Close'].rolling(window=20).std()

# Create a plot
fig = sp.make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Candlestick(x=monthly_sp500.index,
                open=monthly_sp500['Open'],
                high=monthly_sp500['High'],
                low=monthly_sp500['Low'],
                close=monthly_sp500['Close']),
                secondary_y=False)

fig.add_trace(go.Scatter(x=monthly_sp500.index, y=monthly_sp500['MA'], mode='lines', name='Moving Average'),
                secondary_y=True)
fig.add_trace(go.Scatter(x=monthly_sp500.index, y=monthly_sp500['upper_band'], mode='lines', name='Upper Band'),
                secondary_y=True)
fig.add_trace(go.Scatter(x=monthly_sp500.index, y=monthly_sp500['lower_band'], mode='lines', name='Lower Band'),
                secondary_y=True)

# Add rectangular shapes for each cycle
for cycle, details in cycles.items():
    for year in details['years']:
        fig.add_shape(
            type='rect',
            x0=str(year) + '-01-01',
            y0=0,
            x1=str(year+1) + '-01-01',
            y1=1,
            yref='paper',
            fillcolor=details['color'],
            opacity=0.2,
            line_width=0,
        )
        # Use the closing price of the last month of the year as the position for the annotation
        subset = monthly_sp500[monthly_sp500.index.year == year]['Close']
        if not subset.empty:
            y_position = subset.iloc[-1]
            fig.add_annotation(
                x=str(year) + '-06-01',
                y=y_position,
                xref='x',
                yref='y',
                text=cycle,
                showarrow=True,
                arrowhead=1,
                ax=0,
                ay=-10,  # Adjust this to move the annotation closer to the price
                font=dict(color=details['color']),
            )

# Set the title and layout details
fig.update_layout(
    title='S&P 500 with Benner Cycle Indicators, Moving Average, and Bollinger Bands',
    yaxis_title='Price',
    xaxis_title='Year'
)

# Display the figure
fig.show()